In [1]:
import os
import numpy as np
import pickle
import json
import params
import utils
from tqdm import tqdm
from collections import defaultdict

from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [2]:
captions = np.load(r'C:\Users\bldr1\Desktop\TESIS\DATASETS\spanish_train.npy')

In [3]:
def stemming(captions):
    stemmed_dict = defaultdict(list)
    stop_words = set (stopwords.words( 'spanish' ))
    ps = SnowballStemmer("spanish", ignore_stopwords=False)
    unigram_vectorizer = CountVectorizer()
    analyze = unigram_vectorizer.build_analyzer()

    for caption in captions:
        name = caption.get('image_id')
        des = caption.get('caption').lower()

        #remove stopwords
        des_stemmed = []
        for word in analyze(des):
            if word not in stop_words:
                des_stemmed.append(ps.stem(word))
        des_stemmed = ' '.join(des_stemmed)
        
        stemmed_dict[name].append(des_stemmed)
    return stemmed_dict

In [7]:
stemmed_dict = stemming(captions)

In [9]:
stemmed_dict[28231]

['coch negr cerc algui mont biciclet',
 'person biciclet mont part delanter coch',
 'ciclist borros zoom alla nuev merced',
 'ciclist borr va coch negr',
 'hombr mont biciclet alla coch negr estacion']

In [10]:
corpus =[]
for x in stemmed_dict.keys():
    for y in stemmed_dict.get(x):
        corpus.append(y)

### TFIDF

In [13]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2))
transformer = TfidfTransformer(smooth_idf=False)

In [14]:
counts = bigram_vectorizer.fit_transform(corpus)
transformer.fit(counts)

TfidfTransformer(norm='l2', smooth_idf=False, sublinear_tf=False,
         use_idf=True)

In [17]:
utils.savePickle(object=bigram_vectorizer,
                 PATH=r'C:\Users\bldr1\Desktop\TESIS\MODEL\bigram_spanish.pickle')
utils.savePickle(object=transformer, 
                 PATH=r'C:\Users\bldr1\Desktop\TESIS\MODEL\tf_idf_spanish.pickle')

In [18]:
for x in tqdm(stemmed_dict.keys()):
    bigram = bigram_vectorizer.transform(stemmed_dict.get(x))
    tf_idf = transformer.transform(bigram)
    name = r'C:\Users\bldr1\Desktop\TESIS\DESCRIPTORS\TFIDF_SPANISH\TRAIN\\'+str(x)+'.npy'
    np.save(name, tf_idf)

  0%|          | 0/82783 [00:00<?, ?it/s]C:\Users\bldr1\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
100%|██████████| 82783/82783 [24:20<00:00, 56.66it/s]


In [19]:
bigram.shape

(5, 395479)

In [3]:
bigram_vectorizer = utils.importPickle(r'C:\Users\bldr1\Desktop\TESIS\MODEL\bigram_spanish.pickle')
transformer = utils.importPickle(r'C:\Users\bldr1\Desktop\TESIS\MODEL\tf_idf_spanish.pickle')

In [34]:
stop_words = set (stopwords.words( 'spanish' ))
ps = SnowballStemmer("spanish", ignore_stopwords=False)
unigram_vectorizer = CountVectorizer()
analyze = unigram_vectorizer.build_analyzer()

for key in es.keys():
    cap = es[key]['caption'].lower()
    des_stemmed = []
    for word in analyze(cap):
        if word not in stop_words:
            des_stemmed.append(ps.stem(word))
    des_stemmed = ' '.join(des_stemmed)
    counts = bigram_vectorizer.transform([des_stemmed])
    tfidf = transformer.transform(counts)
    es[key]['tfidf'] = tfidf


C:\Users\bldr1\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


### R-TFIDF

In [20]:
r_bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), min_df = 3)
r_transformer = TfidfTransformer(smooth_idf=False)

In [21]:
counts = r_bigram_vectorizer.fit_transform(corpus)
r_transformer.fit(counts)

TfidfTransformer(norm='l2', smooth_idf=False, sublinear_tf=False,
         use_idf=True)

In [22]:
utils.savePickle(object=r_bigram_vectorizer,
                 PATH=r'C:\Users\bldr1\Desktop\TESIS\MODEL\reduced_bigram_spanish.pickle')
utils.savePickle(object=r_transformer, 
                 PATH=r'C:\Users\bldr1\Desktop\TESIS\MODEL\reduced_tf_idf_spanish.pickle')

In [23]:
counts.shape

(414113, 94412)

In [24]:
for x in tqdm(stemmed_dict.keys()):
    bigram = r_bigram_vectorizer.transform(stemmed_dict.get(x))
    tf_idf = r_transformer.transform(bigram)
    name = r'C:\Users\bldr1\Desktop\TESIS\DESCRIPTORS\TFIDF_REDUCED_SPANISH\TRAIN\\'+str(x)+'.npy'
    np.save(name, tf_idf)

  0%|          | 0/82783 [00:00<?, ?it/s]C:\Users\bldr1\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
100%|██████████| 82783/82783 [08:17<00:00, 166.53it/s]


In [36]:
r_bigram_vectorizer = utils.importPickle(r'C:\Users\bldr1\Desktop\TESIS\MODEL\reduced_bigram_spanish.pickle')
r_transformer = utils.importPickle(r'C:\Users\bldr1\Desktop\TESIS\MODEL\reduced_tf_idf_spanish.pickle')

In [37]:
stop_words = set (stopwords.words( 'spanish' ))
ps = SnowballStemmer("spanish", ignore_stopwords=False)
unigram_vectorizer = CountVectorizer()
analyze = unigram_vectorizer.build_analyzer()

for key in es.keys():
    cap = es[key]['caption'].lower()
    des_stemmed = []
    for word in analyze(cap):
        if word not in stop_words:
            des_stemmed.append(ps.stem(word))
    des_stemmed = ' '.join(des_stemmed)
    counts = r_bigram_vectorizer.transform([des_stemmed])
    rtfidf = r_transformer.transform(counts)
    es[key]['rtfidf'] = rtfidf

C:\Users\bldr1\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:1059: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [4]:
c = np.load(r'C:\Users\bldr1\Desktop\TESIS\TMP\test_captions')

In [9]:
list(c.keys()).__len__()

1000

In [13]:
for key in c.keys():
    print(key,';' ,c[key]['caption'])

174390 ; A person on a motor bike on a street.
352507 ; The sun shines through a window into a clean living room with a tile floor.
540816 ; A red and gold Southwest airplane flying above the clouds.
54007 ; A young boy holding a baseball bat and a  stick.
468751 ; Woman in a dress and boots talk on her cell phone while walking.
419235 ; Armed soldiers are running towards a yellow bus.
193946 ; Two monks with umbrella standing on a pavement 
95336 ; A boy eating out of a bowl with bags of donuts 
98853 ; There is a man hitting a tennis ball.
344415 ; Multicolored trains on tracks with people observing them.
518889 ; A sausage on a piece of bread next to different kinds of vegetables.

557107 ; A woman is half naked and laying on her bed while swinging her face to the side.
103904 ; a woman is sitting with an umbrella over her
371365 ; People behind a barricade watch a man ride a motorcycle.
88682 ; A group of people have a picnic on a sunny day.
102805 ; A man holding a white object in

273961 ; A guy is in mid air as he leaps over a skateboard.
456865 ; A large white airplane parked upon a runway
92050 ; A gray block building with a golden decorated clock and the word "Station" on it next to a very tall building.
333772 ; Two cats are laying on the keyboard of a computer. 
324189 ; A piece of a surfboard on top of rocks in a lush green yard.
136021 ; the eleven steps to making a pineapple topped cake
168714 ; A green bus driving through a rural area with trees in the background.
341856 ; A close up of the stems of flowers in a clear vase with water
535809 ; a cat is laying out on a coach
186977 ; The young boy is holding a tennis racquet.
317070 ; A person flying through the air on the kite board.
142890 ; The dark cat is near a keyboard and computer monitor.
190722 ; A laptop and a desktop sitting next to each other.
392392 ; Two motorcycles driving down a freeway through traffic.
18750 ; A tall white fortress surrounded by tall trees.
223019 ; a train stopped at a 

115618 ; A lot of white snow and a couple of skiers.
573094 ; A view of a room that has a Christmas tree that has been decorated.
49060 ; A train sitting next to a loading platform with people standing on it.
63882 ; A colorful bathroom with a pink toilet and tub.
28978 ; Different images of computer accessories such as mice and keyboard.
302030 ; The desk with a laptop has papers strewn all over it.
317939 ; A skier is grabbing his skis during a jump.
502347 ; A plane flies over the hills above a developed shoreline.
278747 ; The train is going down the railroad tracks. 
107389 ; Someone swinging their tennis racket on the tennis court. 
353001 ; A man with his shirt open walking past a group of women in pink dresses with pink umbrellas.
218766 ; A woman in a blackshirt and white skirt playing tennis.
247333 ; A plate full of food on a table with a glass and a bottle.
169347 ; A black and white terrier looks up at the camera. 
259683 ; an image of a night scene walking in rain
372227 

568337 ; Two small white plastic boxes sit on a tiled surface.
533145 ; A hand holding a Nintendo Wii game controller.
576827 ; A beach scene with chairs and an umbrella.
134199 ; A herd of elephants walking through the mud.
46847 ; A flock birds sitting on top of a sandy beach.
471273 ; A teen flying a kite in a hilly area while other people relax nearby.
401892 ; A young boy trying to get his striped kite to fly.
182369 ; A seal is in water by a cement ball.
43614 ; A large white bear standing on a rock.
359801 ; A man who is painting designs on a vase.
45367 ; A woman and a dog sitting with abook
489407 ; These ladies with their identification around their necks are working on their iphones.
222458 ; Entire front yard is filled with snow while people walk around.

15272 ; a beautiful red bud tree full of flowers under a stop light 
61315 ; Small child sitting in front of a table with handmade pizzas.
463932 ; there is a fire hydrant standing on a tall pole
503255 ; A wooden bench si

In [17]:
es = {}
for i, key in enumerate(c.keys()):
    es[key] = {}
    es[key]['caption'] = tr[i]

In [72]:
es['520430']

{'caption': 'Una mujer sonriendo muy grande mientras sostiene un mando de Wii.',
 'rtfidf': <1x94412 sparse matrix of type '<class 'numpy.float64'>'
 	with 13 stored elements in Compressed Sparse Row format>,
 'tfidf': <1x395479 sparse matrix of type '<class 'numpy.float64'>'
 	with 13 stored elements in Compressed Sparse Row format>,
 'w2v': array([-3.42142857e-02,  1.60571429e-02,  3.40285714e-02, -2.70142857e-02,
        -3.82857143e-02, -4.09857143e-02, -1.06285714e-02, -9.22857143e-03,
         1.96000000e-02, -3.67000000e-02,  1.57142857e-02, -1.75000000e-02,
        -3.37571429e-02, -8.55714286e-03, -5.55714286e-03,  1.84714286e-02,
         3.77571429e-02,  1.40285714e-02,  3.88285714e-02,  1.49142857e-02,
        -1.31000000e-02, -1.85714286e-03,  4.12857143e-03,  1.03714286e-02,
        -2.73142857e-02, -3.92571429e-02, -1.84000000e-02, -2.42285714e-02,
         4.40000000e-03,  7.81428571e-03,  3.48571429e-03, -8.78571429e-03,
         3.34142857e-02, -5.97142857e-03,  1.177

In [42]:
c['520430']['caption']

{'caption': 'A women smiling really big while holding a Wii remote.',
 'r-tfidf': <1x91155 sparse matrix of type '<class 'numpy.float64'>'
 	with 11 stored elements in Compressed Sparse Row format>,
 'tfidf': <1x369537 sparse matrix of type '<class 'numpy.float64'>'
 	with 12 stored elements in Compressed Sparse Row format>,
 'w2v': array([ 1.07714286e-02,  1.50000000e-03, -5.45571429e-02, -9.12857143e-03,
        -5.26714286e-02,  3.50285714e-02,  3.85714286e-03,  7.97428571e-02,
        -9.64285714e-03, -2.27000000e-02,  8.07142857e-03, -8.91571429e-02,
         2.37285714e-02,  4.44285714e-02, -4.49428571e-02,  3.39714286e-02,
        -2.24428571e-02, -3.68857143e-02, -4.96571429e-02,  8.50000000e-03,
        -1.31457143e-01, -5.13857143e-02, -3.82428571e-02,  8.67142857e-03,
         5.90000000e-02, -4.85571429e-02,  1.67142857e-02,  7.24857143e-02,
        -7.57142857e-04, -1.51428571e-02,  2.59714286e-02, -1.45714286e-03,
         5.81428571e-02, -6.27142857e-03,  1.88571429e-02,

In [73]:
utils.savePickle(object=es,PATH='TMP/spanish_pickle.pickle')

In [71]:
np.save('TMP/spanish_test.npy',es)

In [40]:
import io

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    #n, d = map(int, fin.readline().split())
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = [float(x) for x in tokens[1:]]
    return data

In [49]:
%%time
fname = 'MODEL/cc.es.300.vec'
vectors = load_vectors(fname)

Wall time: 42min 12s


In [65]:
def applyword2vec(captions, vectors):
    from sklearn.feature_extraction.text import CountVectorizer
    from nltk.corpus import stopwords
    unigram_vectorizer = CountVectorizer()
    analyze = unigram_vectorizer.build_analyzer()
    stop_words = set (stopwords.words( 'spanish' ))
    
    i = 0
    for key in tqdm(captions.keys()):

        caption = captions[key]['caption'].lower()
        
        caption_descriptor = []
        for word in analyze(caption):
            if word not in stop_words:
                if vectors.get(word) is not None:
                    caption_descriptor.append( np.asarray(vectors[word] ))
                else:
                    caption_descriptor.append( np.zeros(300))

        caption_descriptor = np.mean(caption_descriptor, axis=0)
        captions[key]['w2v'] = caption_descriptor
    return captions

In [66]:
es = applyword2vec(fake, vectors)

100%|██████████| 1000/1000 [00:00<00:00, 5280.88it/s]


In [70]:
es['520430']['w2v']

array([-3.42142857e-02,  1.60571429e-02,  3.40285714e-02, -2.70142857e-02,
       -3.82857143e-02, -4.09857143e-02, -1.06285714e-02, -9.22857143e-03,
        1.96000000e-02, -3.67000000e-02,  1.57142857e-02, -1.75000000e-02,
       -3.37571429e-02, -8.55714286e-03, -5.55714286e-03,  1.84714286e-02,
        3.77571429e-02,  1.40285714e-02,  3.88285714e-02,  1.49142857e-02,
       -1.31000000e-02, -1.85714286e-03,  4.12857143e-03,  1.03714286e-02,
       -2.73142857e-02, -3.92571429e-02, -1.84000000e-02, -2.42285714e-02,
        4.40000000e-03,  7.81428571e-03,  3.48571429e-03, -8.78571429e-03,
        3.34142857e-02, -5.97142857e-03,  1.17714286e-02, -2.91142857e-02,
        2.43285714e-02, -1.81714286e-02,  3.45714286e-02,  1.40285714e-02,
       -1.67714286e-02, -1.35142857e-02, -1.54571429e-02,  1.26142857e-02,
       -8.51857143e-02,  2.88857143e-02,  1.35142857e-02, -1.49000000e-02,
       -4.05285714e-02, -1.58714286e-02,  4.14142857e-02,  5.53714286e-02,
        2.41714286e-02, -